# Norm bot ký tự, khoảng trắng, convert low() words

In [4]:
import csv
import json
import pandas as pd
import re
import os

#Tách tên .wav + duration từu .csv gốc tạo thành 1 tuple
def extract_file_info(csv_path):
    """
    Input: File .csv có các trường như file,text,duration
    Output: List tuple (file_name, duration)
    """
    file_info_list = []
    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            file_path = row['file']
            duration = float(row['duration'])
            file_name = file_path.split('/')[-1]
            file_info_list.append((file_name, duration))

    return file_info_list

def get_duration(file_name, file_info_list):
    """
    Trả về duration của file_name từ file_info_list.
    
    Parameters:
    file_name (str): Tên file cần tìm (vd: '542.wav').
    file_info_list (list): Danh sách các tuple chứa tên file và duration.
    Returns:
    float: Thời lượng của file nếu tìm thấy, None nếu không tìm thấy.
    """
    for file, duration in file_info_list:
        if file == file_name:
            return duration
    return None

def extract_and_save_csv(input_csv_path, output_csv_path):
    """
    Trích xuất các trường cần thiết từ file CSV ban đầu và lưu vào file CSV mới.

    Parameters:
    input_csv_path (str): Đường dẫn tới file CSV ban đầu.
    output_csv_path (str): Đường dẫn tới file CSV mới sẽ lưu.

    Returns:
    None
    """
    with open(input_csv_path, newline='', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = ['file', 'text', 'duration']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)

        writer.writeheader()
        for row in reader:
            name_wav = f"replace/{extract_filename(row['file'])}"
            new_row = {
                'file': name_wav,
                'text': row['text_bartpho'],
                'duration': row['duration']
            }
            writer.writerow(new_row)

def remove_punctuation(text):
    return re.sub(r'[^\w\s]',' ', text)

def clean_text(text):
    """ Chuẩn hóa từ bằng cách chuyển về viết thường và xóa bỏ dấu cách (space) dư thừa

    Args:
        text(str): Câu đầu vào
    Returns:
        (str): Câu sau khi được chuẩn hóa
    """
    text = text.lower()
    text = remove_punctuation(text)
    text = ' '.join(text.split())
    return text 

def extract_filename(file_path):
    """
    Input:
        /mnt/driver/pdnguyen/studen_annoted/data_telesale_extract_10_dir/extra_0/wavs/1620.wav
    Output:
        1620.wav
    """
    return os.path.basename(file_path)

## Check exist file

In [4]:
import os
import csv
import tqdm

def check_file_existence(csv_file):
    with open(csv_file, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        
        for row in tqdm.tqdm(reader):
            file_path = row['file']
            if os.path.exists(file_path):
                continue
            else:
                print(f"File does NOT exist: {file_path}")

# Đường dẫn tới file CSV của bạn
csv_file = "/home/pdnguyen/fast_confomer_finetun/finetune-fast-conformer/notebooks/results_csv/20_aug57_10aug_64_total_origin_data.csv"
check_file_existence(csv_file)

2410111it [00:35, 68362.73it/s] 


## Check colums data type

In [5]:
import pandas as pd
def check_data_types(csv_file):
    # Đọc dữ liệu từ file CSV
    df = pd.read_csv(csv_file, dtype=str)  # Đọc toàn bộ dữ liệu dưới dạng chuỗi để xử lý sau

    # Kiểm tra từng dòng
    for index, row in df.iterrows():
        file_value = row['file']
        text_value = row['text']
        duration_value = row['duration']
        
        # Kiểm tra kiểu dữ liệu của cột 'file'
        if not isinstance(file_value, str):
            print(f"Invalid 'file' at row {index + 1}: {file_value}")

        # Kiểm tra kiểu dữ liệu của cột 'text'
        if not isinstance(text_value, str):
            print(f"Invalid 'text' at row {index + 1}: {text_value}")

        # Kiểm tra kiểu dữ liệu của cột 'duration'
        try:
            # Cố gắng chuyển đổi giá trị thành float
            float(duration_value)
        except ValueError:
            print(f"Invalid 'duration' at row {index + 1}: {duration_value}")

# Đường dẫn tới file CSV
check_time = "/home/pdnguyen/fast_confomer_finetun/finetune-fast-conformer/notebooks/results_csv/20_aug57_10aug_64_total_origin_data.csv"
check_data_types(check_time)

## Creat new CSV norm text

In [5]:
import csv
from tqdm import tqdm
def filter_and_clean_csv(input_file, output_file):
    with open(input_file, mode='r', encoding='utf-8') as infile, open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = ['file', 'text', 'duration']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        # Ghi dòng tiêu đề vào file đầu ra
        writer.writeheader()
        for row in tqdm(reader):
            # Lấy các giá trị cần thiết và làm sạch cột 'text_bartpho'
            file_value = row['file']
            text_value = clean_text(row['text'])
            if text_value == None or text_value == '' or text_value == 'none':
                continue
            duration_value = row['duration']
            # Ghi vào file CSV mới
            writer.writerow({'file': file_value, 'text': text_value, 'duration': duration_value})
input_CSV = "/home/pdnguyen/fast_confomer_finetun/finetune-fast-conformer/notebooks/results_csv/train_v2.csv"
output_CSV = '/home/pdnguyen/fast_confomer_finetun/finetune-fast-conformer/notebooks/results_csv/train_v2_v2.csv'
filter_and_clean_csv(input_CSV, output_CSV)

2414921it [00:14, 164123.64it/s]
